In [5]:
#from libs.corpus import openConllu, check_projectivity
import pyconll
import pyconll.util
import torch
from transformers import AutoModel, AutoTokenizer


In [6]:

train = pyconll.load_from_file('data/it_isdt-ud-dev.conllu')
train_prepocesed=[]
for i,sent in enumerate(train):        
    sentence_preprocesed=[]
    for j,token in enumerate(sent):
        if(token.head is not None):
            sentence_preprocesed.append(token)
    train_prepocesed.append(sentence_preprocesed)

for i,sent in enumerate(train_prepocesed):     
    for j,token in enumerate(sent):
        print(token.head)
    print("------")

0
1
4
1
4
7
1
7
------
2
6
6
5
6
0
8
6
11
11
6
13
11
6
------
3
3
5
3
0
9
9
9
5
11
9
13
11
5
------
4
4
4
9
9
5
9
7
0
9
12
10
19
19
14
19
19
19
12
21
19
21
24
22
24
9
------
2
0
2
5
3
8
8
5
2
------
2
0
2
7
7
7
2
2
------
2
3
0
6
6
3
3
------
0
4
4
1
1
------
3
3
0
6
6
3
8
6
3
3
------
2
13
2
3
9
9
9
9
3
9
2
13
0
16
16
13
19
19
16
19
22
19
22
22
22
28
28
25
32
32
32
28
35
35
32
32
13
------
2
9
2
2
7
7
9
7
0
11
9
14
14
11
9
17
9
19
17
21
17
23
21
9
------
2
9
2
7
7
7
2
9
0
13
13
13
9
15
9
18
18
15
21
21
18
26
26
26
26
18
26
38
30
38
35
35
35
35
30
35
38
26
41
41
38
41
9
------
18
1
5
5
18
5
8
18
8
13
13
13
5
13
13
18
18
0
20
18
20
23
18
27
27
27
23
29
27
31
29
33
27
36
36
33
39
39
36
41
39
41
44
39
44
48
48
39
48
51
48
51
51
48
56
48
59
59
56
18
------
10
1
1
5
1
5
1
10
10
0
10
13
11
15
10
15
18
15
21
21
18
24
24
18
15
27
15
30
30
27
30
34
34
30
34
40
40
40
40
15
43
43
40
43
44
48
48
40
51
51
48
55
55
55
51
55
58
55
58
62
62
58
64
62
64
64
58
70
70
58
72
70
10
------
2
8
2
6
6
8
6
0
8


In [21]:
model_name = "dbmdz/bert-base-italian-xxl-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
        
tokens = tokenizer.tokenize('La vita è ')

print(tokens)

input_ids = tokenizer.encode('La vita è', add_special_tokens=True)

print(input_ids)

max_length = 64  # Example max length
padded_input_ids = input_ids + [tokenizer.pad_token_id] * (max_length - len(input_ids))

import torch

input_tensor = torch.tensor([padded_input_ids])
with torch.no_grad():
    outputs = model(input_tensor)

token_index = 3


embeddings = outputs[0]  # Assuming you're using the last layer's output
print(embeddings[0][token_index])


# Assuming you want to extract the embedding of the token at index 2

# Extract embeddings from the final hidden states
hidden_states = outputs.last_hidden_state
token_embedding = hidden_states[0, token_index]

print(token_embedding)

#token_id = tokenizer.convert_tokens_to_ids(token_to_get_id)
#Trasformare l'array di parole in array di ID con la funzione soppra (se con le parole da UNK allora provare con i lemmi)
#creare padded_input_ids
#eseguire il modello con model() e torch 
#implementare l'algorimo di machine learning sugli embedding trovati e sulle varie features(capire come combinare le due)





class Oracle(object):

    def __init__(self) -> None:
        return None


    def score(features):
        return None
        



#creiamo un oggetto Dependencies per salvare le depndencies
class Dependencies(object):
    def __init__(self,n):
        self.n = n
        self.heads = [None] * (n-1)
        self.arcs = []
    
    def add_arc(self, head, child):
        self.heads[child]=head
        self.arcs.append((head,child))

#0:SHIFT 1:RIGHTARC 2:LEFTARC
moves = [0, 1, 2]

class Parser(object):

    model=Oracle()
        
    def transition(move, stack, i, dependencies):
        match move:
            case 0:
                stack.append(i)
                return i+1
            case 1:
                dependencies.add_arc(stack[-2], stack.pop())
                return i
            case 2:
                dependencies.add_arc(stack[-1], stack[-2])
                stack.pop(-2)
                return i
            case _:
                raise "Wrong Move"
            
    def get_valid_moves(i, n, stack_depth):
        moves = []
        if i < n:
            moves.append(0)
        if stack_depth >= 2:
            moves.append(1)
            moves.append(2)
        return moves

    def parsing(self,words,tags):
        if(len(words)!=len(tags)):
            raise "Wrong tag len"
        n=len(words)
        deps=Dependencies(n)
        stack=[0]
        i_buffer=1
        moves=self.get_valid_moves(i,n,len(stack))
        while moves:
            features = ['FeaturesDaScegliere']
            scores = self.model.score(features)
            next_move = max(moves, key=lambda move: scores[move])
            i_buffer = self.transition(next_move, i_buffer, stack, deps)
            moves = self.get_valid_moves(i,n,len(stack))
        return deps

        
    
    

        









['La', 'vita', 'è']
[102, 309, 760, 198, 103]
tensor([-3.9063e-02,  5.8704e-03,  2.3991e-01,  5.0868e-02,  5.8047e-01,
        -6.1092e-01, -3.5590e-01, -2.2224e-01, -5.5669e-01, -3.5191e-01,
        -9.5815e-01, -1.5196e+00,  4.3511e-01,  1.5241e-01,  4.0195e-01,
         1.7193e-01,  1.0694e+00, -1.0936e-01, -9.7706e-01,  8.9866e-02,
        -8.7980e-01,  8.9624e-01, -9.0321e-02,  3.7618e-01, -2.3938e-01,
        -1.6976e+00, -1.1075e+00,  8.7221e-02,  7.7543e-01,  1.1755e+00,
        -4.2746e-01, -1.6892e-01,  8.9258e-01, -2.9968e-01,  3.0602e-01,
        -5.2799e-01, -6.2760e-01,  8.0000e-02, -9.2342e-02, -7.3714e-01,
        -1.6957e-01,  3.4429e-01,  1.9659e-01, -3.0336e-01,  2.5078e-01,
        -1.3579e+00, -5.7441e-01,  8.4614e-01,  2.6777e-01,  6.3693e-03,
         7.7046e-01,  4.2466e-02,  7.2951e-02, -3.8530e-01,  8.3826e-01,
         6.6264e-01,  3.7575e-01, -4.6809e-01,  1.0349e-01,  1.0774e+00,
        -5.3839e-01, -8.6107e-01,  9.7237e-02, -1.0596e+00,  4.1949e-01,
     